In [1]:
import pandas as pd
import spacy
import requests

/opt/anaconda3/envs/naacp-topic-model/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# read gbh rss feed 
df = pd.read_csv("../gbh_rss/search-result-2022-11-08-04-04-19.csv")
clean_df = df.iloc[:, :6]
clean_df

,Type,Label,Headline,Body,Byline,Publish Date
0,Article,Watertown Police Detective Alleges Hostile Wor...,Watertown Police Detective Alleges Hostile Wor...,BOSTON (AP) — A Watertown police detective all...,Associated Press,Wed Nov 18 14:39:02 EST 2020
1,Article,"In 2020, Larissa FastHorse's 'The Thanksgiving...","In 2020, Larissa FastHorse's 'The Thanksgiving...","In late October, Native American playwright <a...",Jill Kaufman,Fri Nov 20 16:34:53 EST 2020
2,Article,In From The Cold: Martha’s Vineyard To Open It...,In From The Cold: Martha’s Vineyard To Open It...,"For the past five winters, three Martha’s Vine...",Jennette Barnes,Tue Nov 24 15:48:23 EST 2020
3,Article,Retail Behemoth Amazon May Be Coming To The Si...,Retail Behemoth Amazon May Be Coming To The Si...,A site plan filed with the city of Worcester’s...,Aaron Schachter,Fri Nov 27 15:11:17 EST 2020
4,Article,Rollins Talking Criminal Justice With Biden Team,Rollins Talking Criminal Justice With Biden Team,While she said &quot;right now I&#39;m stickin...,Michael P. Norton | State House News Service,Mon Nov 30 08:49:08 EST 2020
...,...,...,...,...,...,...
350,Article,Baker recommends pardons of four people,Baker recommends pardons of four people,Gov. Charlie Baker announced pardons of four p...,Jenifer B. McKim,Wed Oct 26 15:29:34 EDT 2022
351,Article,Ketanji Brown Jackson's recusal in Harvard adm...,Ketanji Brown Jackson's recusal in Harvard adm...,When the U.S. Supreme Court announces its deci...,Diane Adame,Mon Oct 31 15:15:03 EDT 2022
352,Article,It's now illegal in Mass. to throw out used je...,It's now illegal in Mass. to throw out used je...,It&#39;s now against the law in Massachusetts ...,Craig LeMoult,Tue Nov 01 09:58:52 EDT 2022
353,Article,Salem takes in record numbers of Halloween tou...,Salem takes in record numbers of Halloween tou...,Saying that Halloween in Salem is “a big deal”...,James Bennett II,Mon Oct 31 16:32:14 EDT 2022


In [20]:
def get_locations(article_text):
    """
    get location names from article using NER 
    input: article_text as a string, aggregate of h1, h2, lede, and body
    returns: locations - set of tuples of (NAME, 'GPE')
    """
    # get locations using NER  
    nlp = spacy.load("en_core_web_lg")
    doc = nlp(article_text)

    # get the locations only, remove duplicates from results 
    locations = set([(X.text, X.label_) for X in doc.ents if X.label_ == 'GPE' or X.label_ == 'LOC' or X.label_ == 'FAC']) # or X.label_ == 'LOC' or X.label_ == 'FAC' or X.label_ == 'ORG'
    
    orgs = set([(X.text, X.label_) for X in doc.ents if X.label_ == 'ORG'])
    return locations, orgs

In [49]:
test = "Long closed sporting-goods store in BU East MBTA could be replaced by six-story residential building. A developer has filed plans to replace the old City Sports at 1035 Commonwealth Ave. in Allston with a six-story, 55-unit residential building with retail space on the ground floor. In a filing with the BPDA, developers BGI Homes of Brighton and D2 Development of Roxbury say the $15-million project would have units ranging from studios to three bedrooms. The plans do not specify whether the units would be rented as apartments or sold as condos. Seven of the units would be offered as affordable. The building, right on the B Line and the 57 bus route, would have no parking spaces for residents but two for the retail space. The filing notes that Fenway Park is a 29-minute walk away. Residents would have easier access to Thai food - at Brown Sugar next door. The developer expect the units will be very attractive to those seeking an active neighborhood within the urban context of Boston. City Sports closed in 2015."

In [50]:
#!python3 -m spacy download en_core_web_lg
i = 15
text = df['Label'][i] + df['Headline'][i] + df['Body'][i]
loc, org = get_locations(test)
print(loc)
print()
print(org)

{('Allston', 'GPE'), ('Boston', 'GPE'), ('Fenway Park', 'FAC'), ('Brown Sugar', 'FAC')}

{('D2 Development of Roxbury', 'ORG'), ('City Sports', 'ORG'), ('BGI Homes of Brighton', 'ORG'), ('the B Line', 'ORG'), ('BU East MBTA', 'ORG')}


In [45]:
print(test)

Long closed sporting-goods store in Packards Corner could be replaced by six-story residential building. A developer has filed plans to replace the old City Sports at 1035 Commonwealth Ave. in Allston with a six-story, 55-unit residential building with retail space on the ground floor. In a filing with the BPDA, developers BGI Homes of Brighton and D2 Development of Roxbury say the $15-million project would have units ranging from studios to three bedrooms. The plans do not specify whether the units would be rented as apartments or sold as condos. Seven of the units would be offered as affordable. The building, right on the B Line and the 57 bus route, would have no parking spaces for residents but two for the retail space. The filing notes that Fenway Park is a 29-minute walk away. Residents would have easier access to Thai food - at Brown Sugar next door. The developer expect the units will be very attractive to those seeking an active neighborhood within the urban context of Boston.